In [13]:
from eurec4a_environment import source_data
import matplotlib.pyplot as plt

import datetime
import numpy as np
import xarray as xr

import requests
requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS = 'AES256-SHA'

Quesitons for Ilya:

- Why is `Altitude` 2D? Looks like the values don't change with time so could be 1D?
- Fill value isn't set, but quite a few values ~10e32
- Found large negative value for water vapour mixing ratio (-297808.47) at 32640 seconds past midnight on the 2020-01-11 at 80m altitude)

In [14]:
cat = source_data.get_intake_catalog()


In [15]:
cat.barbados.bco.CORAL_LIDAR.describe()

{'name': 'CORAL_LIDAR',
 'container': 'xarray',
 'plugin': ['opendap'],
 'description': 'Observations made with the CORAL LIDAR at BCO',
 'direct_access': 'forbid',
 'user_parameters': [{'name': 'version',
   'description': 'dataset version to use',
   'type': 'str',
   'default': '2020.07.27'},
  {'name': 'date',
   'description': 'date of observation',
   'type': 'datetime',
   'min': Timestamp('2020-01-09 00:00:00'),
   'max': Timestamp('2020-02-29 00:00:00'),
   'default': Timestamp('2020-01-09 00:00:00')},
  {'name': 'dt',
   'description': 'time resolution',
   'type': 'int',
   'allowed': [10, 20, 60],
   'default': 10},
  {'name': 'content_type',
   'description': 'dataset-content',
   'type': 'str',
   'allowed': ['b', 't'],
   'default': 'b'}],
 'metadata': {'maintainer': 'Leif Denby <l.c.denby[at]leeds.ac.uk>',
  'documentation': 'https://wiki.mpimet.mpg.de/doku.php?id=analysis:data:bco:ramanlidars:raman-lidar-coral',
  'tags': ['water-vapour profiles', 'temperature profiles

In [16]:
datasets = []
for n in range(50):
    date = datetime.datetime(year=2020, month=1, day=9) + datetime.timedelta(days=n)
    try:
        ds = cat.barbados.bco.CORAL_LIDAR(date=date, content_type="t").to_dask()
    except:
        raise
    ds['time'] = ('Time'), np.datetime64(date) + ds.Time.values.astype('timedelta64[s]')
    ds = ds.swap_dims(dict(Time='time'))
    datasets.append(ds)

SSLError: HTTPSConnectionPool(host='observations.ipsl.fr', port=443): Max retries exceeded with url: /thredds/dodsC/EUREC4A/BARBADOS/BCO/Raman_Lidar_CORAL/lowResolution/version_2020.07.27/T10min/nc/coral_200109_0002_0000_t.nc.dds (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))

In [ ]:
datasets

In [ ]:
datasets[0].WaterVaporMixingRatio.max().compute()

In [ ]:
ds_all = xr.concat(datasets, dim='time')
ds_all
del ds_all.attrs['_NCProperties']

In [ ]:
ds_all.to_netcdf('coral.nc')

In [ ]:
ds = xr.open_dataset('coral.nc')
ds

In [ ]:
ds['Altitude'] = ds.Altitude.isel(time=0)

In [ ]:
ds['WaterVaporMixingRatio'] = ds.WaterVaporMixingRatio.where(ds.WaterVaporMixingRatio < 1.0e32, other=np.nan)

In [ ]:
da_q_coral = ds.WaterVaporMixingRatio
da_q_coral = da_q_coral.where((da_q_coral >= 0.0)*(da_q_coral < 20.0), np.nan)
ds['WaterVaporMixingRatio'] = da_q_coral

In [ ]:
ds = ds[['WaterVaporMixingRatio', 'Altitude']]

In [ ]:
import s3fs
import xarray as xr
import yaml
import os
from pathlib import Path


s3_access_key = "***"
s3_secret_key = "***"

def add_cat_entry(ds, remote_path, description):

    fs = s3fs.S3FileSystem(
        anon=False,
        key=s3_access_key,
        secret=s3_secret_key,
        client_kwargs={"endpoint_url": "https://minio.denby.eu"},
    )

    cat_entry = {
        "description": description,
        "driver": "zarr",
        "args": {
            "urlpath": f"simplecache::s3://eurec4a-environment/{remote_path}",
            "consolidated": True,
            "storage_options": {
                "s3": {
                    "anon": True,
                    "client_kwargs": {
                        "endpoint_url": "https://minio.denby.eu"
                    }
                }
            }
        }
    }
    mapper = fs.get_mapper(f"eurec4a-environment/{remote_path}")
    print(f"eurec4a-environment/{remote_path}")

    ds.to_zarr(mapper, consolidated=True)

    print(yaml.dump(cat_entry, default_flow_style=False))
    
add_cat_entry(ds=ds, remote_path="bco/coral", description="CORAL LIDAR at BCO")

In [ ]:
datasets[0].Time

In [ ]:
ds_all